# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [0]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

## Get Required Files from Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
#Set your project path 
project_path =  '/content/drive/My Drive/ML_Project/NLP/NLP-2 Sarcasm Detection/'

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [0]:
import pandas as pd
import os

data = pd.read_json(os.path.join(project_path,'Sarcasm_Headlines_Dataset.json'),lines=True)

In [0]:
for i in range(0,11):
  print(data['headline'][i])

former versace store clerk sues over secret 'black code' for minority shoppers
the 'roseanne' revival catches up to our thorny political mood, for better and worse
mom starting to fear son's web series closest thing she will have to grandchild
boehner just wants wife to listen, not come up with alternative debt-reduction ideas
j.k. rowling wishes snape happy birthday in the most magical way
advancing the world's women
the fascinating case for eating lab-grown meat
this ceo will send your kids to school, if you work for his company
top snake handler leaves sinking huckabee campaign
friday's morning email: inside trump's presser for the ages
airline passengers tackle man who rushes cockpit in bomb threat


**OBSERVATION** :
As seen in the above console, the text in 'headline' needs to be formatted.
Special characters and numbers needs to be removed from the text

In [0]:
import re
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
def cleanData(text):
  text = re.sub(r'\d+', '', text)
  text = "".join([char for char in text if char not in string.punctuation])
  #text = " ".join([word for word in text.split(' ') if word not in stopwords]) 
  #--> Model performs better without removing stop words, 
  #    since words like 'not' will get removed which are essential for classification
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
data['headline']=data['headline'].apply(cleanData)

## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [0]:
data.drop('article_link',inplace=True,axis=1)

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [0]:
maxlen = max([len(text) for text in data['headline']])

#**## Modelling**

## Import required modules required for modelling.

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [0]:
max_features = 10000
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [0]:
tokenizer = Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ', char_level=False)

tokenizer.fit_on_texts(data['headline'])

# Define X and y for your model.

In [0]:
X = tokenizer.texts_to_sequences(data['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(data['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [0]:
num_words=len(tokenizer.word_index)

#**## Word Embedding**

## Get Glove Word Embeddings

In [0]:
glove_file = "/content/drive/My Drive/ML_Project/NLP/NLP-1 Sentiment Classification/glove.6B.zip"

In [0]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [0]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [0]:
embedding_matrix = np.zeros((num_words+1, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [0]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

input_layer = Input(shape=(maxlen,),dtype=tf.int64)
embed = Embedding(embedding_matrix.shape[0],output_dim=200,weights=[embedding_matrix],input_length=maxlen, trainable=True)(input_layer)
lstm=Bidirectional(LSTM(128))(embed)
drop=Dropout(0.3)(lstm)
dense =Dense(100,activation='relu')(drop)
out=Dense(2,activation='softmax')(dense)


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [0]:
batch_size = 100
epochs = 5

model = Model(input_layer,out)

model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 240, 200)          5533600   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               336896    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               25700     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 5,896,398
Trainable params: 5,896,398
Non-trainable params: 0
___________________________________________________

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [0]:
model.fit(X_train,y_train,batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/5
214/214 [==============================] - 19s 88ms/step - loss: 0.4498 - accuracy: 0.7803
Epoch 2/5
214/214 [==============================] - 19s 87ms/step - loss: 0.2659 - accuracy: 0.8921
Epoch 3/5
214/214 [==============================] - 19s 86ms/step - loss: 0.1745 - accuracy: 0.9331
Epoch 4/5
214/214 [==============================] - 18s 86ms/step - loss: 0.1134 - accuracy: 0.9583
Epoch 5/5
214/214 [==============================] - 18s 86ms/step - loss: 0.0796 - accuracy: 0.9693


In [0]:
test_pred = model.predict(np.array(X_test), verbose=1)

167/167 [==============================] - 2s 11ms/step


In [0]:
test_pred = [1 if j>i else 0 for i,j in test_pred]

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, test_pred)

array([[2671,  360],
       [ 372, 1939]])

In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_pred))
# Accuracy can still be increased by increasing the number of epochs

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      3031
           1       0.84      0.84      0.84      2311

    accuracy                           0.86      5342
   macro avg       0.86      0.86      0.86      5342
weighted avg       0.86      0.86      0.86      5342

